In [1]:
import os
import xarray as xr
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, minmax_scale

import holodecml.ml_utils as ml


In [7]:
path_data = "/glade/p/cisl/aiml/ggantos/holodec/ft_rad_bidis/"
# path_data = "/glade/p/cisl/aiml/ggantos/holodec/ft_rad_bidis_z/"
# path_data = "/glade/p/cisl/aiml/ggantos/holodec/ft_rad_bidis_z_realimag/"
fns = [x for x in os.walk(path_data)][0][2]
fn_train = [x for x in fns if 'training' in x][0]
fn_valid = [x for x in fns if 'validation' in x][0]
fn_test = [x for x in fns if 'test' in x][0]

batch_size = 256
label_variable = 'histogram'  # data variable used labels for training
input_variable = 'input_image'  # data variable used as an input


In [8]:
with xr.open_dataset(path_data+fn_train, chunks={'hologram_number': batch_size}) as ds:
    print("Loading TRAINING dataset")

    train_labels = ds[label_variable]
    print(f"\n\ttrain_labels.shape:{train_labels.shape}")
    output_scaler = ml.MinMaxScalerX(train_labels)
    scaled_train_labels = output_scaler.fit_transform(train_labels)
    print(f"\n\tscaled_train_labels.shape:{scaled_train_labels.shape}")

    if len(ds[input_variable].dims) == 4:
        train_data = ds[input_variable].transpose('hologram_number','xsize','ysize','input_channels')
    elif len(ds[input_variable].dims) == 3:
        train_data = ds[input_variable].transpose('hologram_number','rsize','input_channels')
    print(f"\n\ttrain_data.shape:{train_data.shape}")
    input_scaler = ml.MinMaxScalerX(train_data)
    scaled_train_input = input_scaler.fit_transform(train_data)
    print(f"\n\tscaled_train_input.shape:{scaled_train_input.shape}")


Loading TRAINING dataset

	train_labels.shape:(5000, 19, 1)

	scaled_train_labels.shape:(5000, 19, 1)

	train_data.shape:(5000, 600, 1)

	scaled_train_input.shape:(5000, 600, 1)


In [9]:
scaled_train_labels.values

array([[[0.04878049],
        [0.19512195],
        [0.24390244],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.31707317],
        [0.41463415],
        [0.17073171],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.02439024],
        [0.14634146],
        [0.34146341],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.17073171],
        [0.56097561],
        [0.46341463],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.07317073],
        [0.2195122 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.14634146],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]])

In [4]:
with xr.open_dataset(path_data+fn_valid, chunks={'hologram_number': batch_size}) as ds:
    print("Loading VALIDATION dataset")

    valid_labels = ds[label_variable]
    print(f"\n\tvalid_labels.shape:{valid_labels.shape}")
    output_scaler = ml.MinMaxScalerX(valid_labels)
    scaled_valid_labels = output_scaler.fit_transform(valid_labels)
    print(f"\n\tscaled_valid_labels.shape:{scaled_valid_labels.shape}")

    if len(ds[input_variable].dims) == 4:
        valid_data = ds[input_variable].transpose('hologram_number','xsize','ysize','input_channels')
    elif len(ds[input_variable].dims) == 3:
        valid_data = ds[input_variable].transpose('hologram_number','rsize','input_channels')
    print(f"\n\tvalid_data.shape:{valid_data.shape}")
    input_scaler = ml.MinMaxScalerX(valid_data)
    scaled_valid_input = input_scaler.fit_transform(valid_data)
    print(f"\n\tscaled_valid_input.shape:{scaled_valid_input.shape}")

Loading VALIDATION dataset

	valid_labels.shape:(1000, 19, 1)

	scaled_valid_labels.shape:(1000, 19, 1)

	valid_data.shape:(1000, 600, 1)

	scaled_valid_input.shape:(1000, 600, 1)


In [ ]:
    mod = Sequential()
    mod.add(Input(shape=scaled_train_input.shape[1:]))


In [ ]:
    history = mod.fit(scaled_train_input.values,
                  scaled_train_labels.values, 
                  batch_size=settings['batch_size'], epochs=settings['num_epochs'], verbose=1,
                  validation_data=(scaled_valid_input.values,scaled_val_labels.values))
